<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/CART_Supervised_Classification_ESRI_land_cover_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

how to convert landsat data into clarssification using CART model of machine learning in python script

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ROAX6kbNpR99L-4cDk2yFpvGSYYqxvMPhKxIMpOk-kQ&tc=1L40oXiZmMwRK6jiCG1g1k9vLHGflc9UfrfaptjkbWg&cc=zLdvLPb3Aju_re3s-nbkFK3hI_sPeUncr8AC2iEUAOY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn2GaJGsJEEi7eLmkNQv8cBPPjP-hVSnYg8evep30oeSbP40lP-AKc

Successfully saved authorization token.


In [23]:
#ROI
roi = ee.Geometry.Point([38.5914, 7.71])
location = roi.centroid().coordinates().getInfo()[::-1]

In [24]:
# Load Landsat 8 data
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
.filterDate('2021-01-01', '2021-12-31') \
.filterBounds(roi) \
.sort('CLOUD_COVER') \
.first()

In [30]:
#Import ESRI Data
land_cover = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").mosaic().clip(image.geometry())
label = 'b1'

In [31]:
# Sample the input imagery to get a FeatureCollection of training data.
sample = image.addBands(land_cover).sample(**{
  'region': image.geometry(),
  'numPixels': 1000,
  'seed': 0
})

#This step creates a random sample of 1000 pixels from the input image within the specified region (image.geometry()). Bands from land_cover are added to each pixel in the sample.


In [33]:
# Add a random value field to the sample and use it to approximately split 80%
# of the features into a training set and 20% into a validation set.
sample = sample.randomColumn() #A random column is added to the sample, which will be used later to split the data into training and validation sets.


trainingSample = sample.filter('random <= 0.8')
validationSample = sample.filter('random > 0.8')
#The samples are split into approximately 80% for training (trainingSample) and 20% for validation (validationSample) based on the random column values.


In [34]:
# Train a 10-tree random forest classifier from the training sample.
trainedClassifier = ee.Classifier.smileRandomForest(10).train(**{
  'features': trainingSample,
  'classProperty': label,
  'inputProperties': bands
})

#A random forest classifier with 10 trees is trained using the features from the training sample (trainingSample). The classProperty parameter specifies the property containing the class labels, and inputProperties specifies the input bands.

In [29]:
# Classify the reflectance image from the trained classifier.
model = image.classify(trainedClassifier)

In [35]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def addLayer (self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.addLayer = addLayer

# Create the map object.
Map = folium.Map(location=location, zoom_start = 8)

# Add base map
basemaps = {'Google Satellite Hybrid': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite Hybrid',
overlay = True,
control = True
)}
basemaps['Google Satellite Hybrid'].add_to(Map)

In [36]:
# Define a dictionary which will be used to make legend and visualize image on map
dict = {
  "names": [
    "Water",
    "Trees",
    "Grass",
    "Flooded Vegetation",
    "Crops",
    "Scrub/Shrub",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds"
  ],

  "colors": [
    "#1A5BAB",
    "#358221",
    "#A7D282",
    "#87D19E",
    "#FFDB5C",
    "#EECFA8",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8"
  ]}

# Add land cover to the map object.
Map.addLayer(lc, {'min':1, 'max':10, 'palette':dict['colors']}, 'ESRI LULC 10m')
Map.addLayer(model, {'min':1, 'max':10, 'palette':dict['colors']},'Classified 2021')

visParamsTrue = {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}
Map.addLayer(image, visParamsTrue, "Landsat 2021")

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
display(Map)